In [1]:
#imports
import pandas as pd 
from pandas import DataFrame
from slugify import slugify
import requests
from tqdm.notebook import tqdm
from time import sleep
from bs4 import BeautifulSoup as bs
import json
from datetime import datetime
import webbrowser
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
from time import sleep

In [333]:
#getting both csv files 
movie_details = pd.read_csv('/Users/armaan/src/bbox/mdata/thenum/movie_details.csv') 
imdb_movies = pd.read_csv('/Users/armaan/src/bbox/mdata/imdb/imdb_movies.csv') 
#converting names to lower and some cols to int
movie_details['name_lower'] = movie_details['name'].str.lower()
imdb_movies['year'] = pd.to_numeric(imdb_movies.startYear, errors='coerce')
imdb_movies['primaryTitle_lower'] = imdb_movies['primaryTitle'].str.lower()
imdb_movies['runtimeMinutes'] = pd.to_numeric(imdb_movies.runtimeMinutes, errors='coerce')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [334]:
#normalization
def normalize(x):
    if x:
        #for star wars
        x = x.replace('-ep-', '-episode-')
        #for harry potter
        x = x.replace('part-ii', 'part-2')
        x = x.replace('part-i', 'part-1')
        #for e.t. only!
        if x.startswith('et-'):
            x = x.replace('et-', 'e-t-')
        #for rise of skywalker only!
        if x == 'star-wars-the-rise-of-skywalker':
            x = "star-wars-episode-ix-the-rise-of-skywalker"
    return x

#find duplicates
def duplicate_scraper(imdb_id):
    r = requests.get("https://www.imdb.com/title/{}/".format(imdb_id))
    soup = bs(r.text)
    box_office = soup.find("div",{"id": "titleDetails"})
    all_h4 = box_office.findAll("h4")
    drop = True
    for h4 in all_h4:
        if h4.string == "Gross USA:":
            drop = False
            break
    return drop

In [335]:
#sluggifying and normalizing all names
movie_details['name_slug'] = movie_details.name_lower.apply(slugify)
imdb_movies['name_slug'] = imdb_movies.primaryTitle_lower.apply(slugify)
movie_details['name_slug'] = movie_details.name_slug.apply(normalize)
#creating new df with updated values
new_df = pd.merge(movie_details, imdb_movies, how ='inner', right_on = ['name_slug', 'year'], left_on = ['name_slug', 'year'])

In [336]:
#find duplicates and get rid of them
duplicates = new_df[new_df.name.duplicated()]
for index, row in tqdm(duplicates.iterrows()):
    match = new_df[(new_df['name'] == row["name"]) & (new_df['year'] == row["year"])]
    if len(match) > 1:
        imdb_ids = match["tconst"]
        for imdb_id in imdb_ids:
            drop = duplicate_scraper(imdb_id)
            if drop == True:
                new_df.drop( new_df[ new_df['tconst'] == imdb_id ].index , inplace=True)

In [337]:
#drops
new_df = new_df[new_df.tconst != 'tt11861230']
new_df = new_df[new_df.tconst != 'tt4064028']

new_df.drop(["endYear", "isAdult", "titleType", "startYear", "primaryTitle_lower", "runtimeMinutes", "genres", "name_lower", "primaryTitle", "originalTitle"], axis=1, inplace = True)

In [338]:
#drop movies with missing information
new_df
new_df2 = new_df.dropna(subset=['domestic_box_office', 'international_box_office', 'worldwide_box_office', 'production_budget', 'opening_weekend'])
new_df = new_df2
new_df[:1137]


,name,year,domestic_box_office,international_box_office,worldwide_box_office,opening_weekend,legs,production_budget,theater_counts,mpaa_rating,running_time,genre,production_method,creative_type,production_companies,synopsis,link,name_slug,tconst
0,Star Wars Ep. VII: The Force Awakens,2015,936662225,1131561399.00,2068223624.00,247966675.00,3.78,306000000.00,9.20,PG-13,136.00,Adventure,Animation/Live Action,Science Fiction,Lucasfilm,"Rey, a scavenger from the planet Jakku, and re...",https://www.the-numbers.com/movie/Star-Wars-Ep...,star-wars-episode-vii-the-force-awakens,tt2488496
1,Avengers: Endgame,2019,858373000,1939427564.00,2797800564.00,357115007.00,3.78,400000000.00,8.10,PG-13,181.00,Action,Animation/Live Action,Super Hero,Marvel Studios,The grave course of events set in motion by Th...,https://www.the-numbers.com/movie/Avengers-End...,avengers-endgame,tt4154796
2,Avatar,2009,760507625,2028193712.00,2788701337.00,77025481.00,3.78,237000000.00,13.20,PG-13,162.00,Action,Animation/Live Action,Science Fiction,Dune Entertainment,"Jake Sully is a wounded ex-marine, thrust into...",https://www.the-numbers.com/movie/Avatar#tab=s...,avatar,tt0499549
3,Black Panther,2018,700059566,646043810.00,1346103376.00,202003951.00,3.78,200000000.00,9.90,PG-13,120.00,Action,Live Action,Super Hero,Marvel Studios,"King T'Challa returns home to the reclusive, t...",https://www.the-numbers.com/movie/Black-Panthe...,black-panther,tt1825683
4,Avengers: Infinity War,2018,678815482,1369544272.00,2048359754.00,257698183.00,3.78,300000000.00,7.70,PG-13,156.00,Action,Animation/Live Action,Super Hero,Marvel Studios,As the Avengers and their allies have continue...,https://www.the-numbers.com/movie/Avengers-Inf...,avengers-infinity-war,tt4154756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325,The Wedding Planner,2001,60400856,34327673.00,94728529.00,13510293.00,3.78,28000000.00,6.50,NaN,nan,NaN,NaN,NaN,NaN,NaN,https://www.the-numbers.com/movie/Wedding-Plan...,the-wedding-planner,tt0209475
1326,Burn After Reading,2008,60355347,107812451.00,168167798.00,19128001.00,3.78,37000000.00,5.30,R,97.00,Black,Live Action,Contemporary Fiction,Relativity Media,NaN,https://www.the-numbers.com/movie/Burn-After-R...,burn-after-reading,tt0887883
1327,Why Him?,2016,60323786,57101370.00,117425156.00,11002986.00,3.78,38000000.00,4.40,R,111.00,Comedy,Live Action,Contemporary Fiction,21 Laps Entertainment,"Over the holidays, Ned, an overprotective but ...",https://www.the-numbers.com/movie/Why-Him#tab=...,why-him,tt4501244
1328,Blockers,2018,60311495,34212286.00,94523781.00,20556350.00,3.78,21000000.00,4.70,R,102.00,Comedy,Live Action,Contemporary Fiction,Point Grey,When three parents discover their daughters' p...,https://www.the-numbers.com/movie/Blockers#tab...,blockers,tt2531344


In [339]:
#get poster path and add it to new col in df
poster_link_list = []
for p in tqdm(new_df['tconst']):
    tmdb = requests.get("https://api.themoviedb.org/3/movie/{}?api_key=a6837e56522d89b354d01b07e393bebc&language=en-US".format(p))
    try:
        poster_path = tmdb.json()['poster_path']
        poster_link = "https://image.tmdb.org/t/p/w300_and_h450_bestv2/{}".format(poster_path)
    except:
        if p:
            print ("no poster for " + p)
    poster_link_list.append(poster_link)

new_df["poster_path"] = poster_link_list

In [340]:
#get similar movies
def get_matching_similar(df_movies, simlist, how_many=5):
    matching_movies = []
    result_cnt = min(len(simlist), how_many)
    for s in simlist:
        try:
            title = s['title']
            release = int(s['release_date'][:4])
            if len(df_movies[(df_movies.name == title) & (df_movies.year == release)]) == 1:
                poster_path = new_df.loc[new_df['name'] == title, 'poster_path'].values[0]
                name = new_df.loc[new_df['name'] == title, 'name'].values[0]
                movie_link = "{}^{}".format(title, release)
                matching_movies.append({'name': name, 'link': movie_link, 'poster' : poster_path})
                
                if len(matching_movies) >= how_many:
                    break
        except Exception as ex:
            print(ex)

            
    return matching_movies


similar_movies_list = []
x = 0
for t in tqdm(new_df['tconst']):
    turl = "https://api.themoviedb.org/3/movie/{}/similar?api_key=a6837e56522d89b354d01b07e393bebc&language=en-US&page=1".format(t)
    tmdb = requests.get(turl)
    if 'results' in tmdb.json():
        sim_list = tmdb.json()['results']
    else:
        sim_list = []
    similar_movies_list.append(get_matching_similar(new_df, sim_list, how_many=5))
#     x = x+1
#     if x > 3:
#         break
        
new_df["similar_movies"] = similar_movies_list

'release_date'
'release_date'
'release_date'
invalid literal for int() with base 10: ''
'release_date'
invalid literal for int() with base 10: ''
invalid literal for int() with base 10: ''
invalid literal for int() with base 10: ''
invalid literal for int() with base 10: ''
'release_date'
invalid literal for int() with base 10: ''
invalid literal for int() with base 10: ''
'release_date'
invalid literal for int() with base 10: ''
invalid literal for int() with base 10: ''
'release_date'
invalid literal for int() with base 10: ''
invalid literal for int() with base 10: ''



In [341]:
#scrape for imdb score
imdb_scores = []
x = 0
for t in tqdm(new_df['tconst']):
    try:
        r = requests.get("https://www.imdb.com/title/{}/".format(t))
        soup = bs(r.text)
        score_section = soup.find("span", {"itemprop": "ratingValue"})
        score = score_section.string
    #     x = x+1
    #     if x > 3:
    #         break
    except Exception as ex:
        score = " "
        print(ex)
        print(t)
    imdb_scores.append(score)
     
new_df["imdb_score"] = imdb_scores

In [342]:
#get metascore (next use - july26)
# meta_scores = []
# x = 0
# for t in tqdm(new_df['tconst']):
#     url = "http://www.omdbapi.com/?apikey=581c8a90&i={}".format(t)
#     omdb = requests.get(url)
#     info = omdb.json()
#     if info['Metascore']:
#         meta_score = info['Metascore']
#     else:
#         meta_score = "N/A"
#     meta_scores.append(meta_score)
# #     x = x+1
# #     if x > 3:
# #         break


# new_df["meta_score"] = meta_scores

In [343]:
#make score
bbox_df = new_df2

bbox_df['bb_profit'] = bbox_df.worldwide_box_office - bbox_df.production_budget
bbox_df['bb_profit_multiple'] = bbox_df.worldwide_box_office/bbox_df.worldwide_box_office

bbox_df['bb_profit_score'] = (bbox_df.bb_profit / bbox_df.bb_profit.max()) * 100
bbox_df['bb_profit_multiple_score'] = (bbox_df.bb_profit_multiple / bbox_df.bb_profit_multiple.max()) * 100

bbox_df['bb_score'] = (bbox_df['bb_profit_score']*0.7 + bbox_df['bb_profit_multiple_score']*0.3)

In [344]:
#find box office shares

bbox_df["domestic_share"] = bbox_df.domestic_box_office / bbox_df.worldwide_box_office * 100
bbox_df["international_share"] = bbox_df.international_box_office / bbox_df.worldwide_box_office * 100
bbox_df.T

,0,1,2,3,4,5,6,7,8,9,...,2361,2363,2367,2368,2369,2371,2372,2373,2375,2376
name,Star Wars Ep. VII: The Force Awakens,Avengers: Endgame,Avatar,Black Panther,Avengers: Infinity War,Titanic,Jurassic World,The Avengers,Star Wars Ep. VIII: The Last Jedi,Incredibles 2,...,Cradle 2 the Grave,Hostage,Man on the Moon,Hot Pursuit,Thirteen Days,The 6th Day,Concussion,Courageous,Monster,The Doors
year,2015,2019,2009,2018,2018,1997,2015,2012,2017,2018,...,2003,2005,1999,2015,2000,2000,2015,2011,2003,1991
domestic_box_office,936662225,858373000,760507625,700059566,678815482,659363944,652270625,623357910,620181382,608581744,...,34657731,34636443,34580635,34580201,34566746,34543701,34531832,34522221,34469210,34416893
international_box_office,1131561399.00,1939427564.00,2028193712.00,646043810.00,1369544272.00,1548844451.00,1018130012.00,891742301.00,712358507.00,634223615.00,...,21777211.00,43000000.00,12827000.00,11091311.00,31987801.00,61481197.00,15831958.00,663663.00,29731111.00,3702.00
worldwide_box_office,2068223624.00,2797800564.00,2788701337.00,1346103376.00,2048359754.00,2208208395.00,1670400637.00,1515100211.00,1332539889.00,1242805359.00,...,56434942.00,77636443.00,47407635.00,45671512.00,66554547.00,96024898.00,50363790.00,35185884.00,64200321.00,34420595.00
opening_weekend,247966675.00,357115007.00,77025481.00,202003951.00,257698183.00,28638131.00,208806270.00,207438708.00,220009584.00,182687905.00,...,16521486.00,10214734.00,7515585.00,13942258.00,46668.00,13020883.00,10513749.00,9112839.00,86831.00,9151800.00
legs,3.78,3.78,3.78,3.78,3.78,3.78,3.78,3.78,3.78,3.78,...,3.78,3.78,3.78,3.78,3.78,3.78,3.78,3.78,3.78,3.78
production_budget,306000000.00,400000000.00,237000000.00,200000000.00,300000000.00,200000000.00,215000000.00,225000000.00,200000000.00,200000000.00,...,25000000.00,75000000.00,52000000.00,35000000.00,80000000.00,82000000.00,35000000.00,2000000.00,5000000.00,40000000.00
theater_counts,9.20,8.10,13.20,9.90,7.70,26.80,8.50,8.70,6.70,9.10,...,3.70,5.20,4.60,3.80,5.20,4.30,3.40,8.20,8.80,6.60
mpaa_rating,PG-13,PG-13,PG-13,PG-13,PG-13,PG-13,PG-13,PG-13,PG-13,PG,...,NaN,R,NaN,PG-13,PG-13,NaN,PG-13,PG-13,R,R


In [345]:
#downloads df to csv
bbox_df = bbox_df[~bbox_df.worldwide_box_office.isnull()]
bbox_df.to_csv('/Users/armaan/src/bbox/mdata/final_result/master_table.csv', index=False)